In [1]:
import numpy as np
import numpy.matlib
import matplotlib
%matplotlib inline
import h5py
import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
# from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
#                                                   mark_inset)
import sys,os
import importlib
# from multiprocessing.dummy import Pool as ThreadPool
import sklearn
import sklearn.decomposition
import sklearn.preprocessing
import datetime
import seaborn as sns
import pandas as pd
import scipy
import scipy.spatial
import dReLU_Regression
import Gaussian_Regression
import scipy.fftpack
# import xarray as xr
# import statsmodels.api as sm
# from statsmodels.tsa.api import VAR
from tqdm import tqdm
import scipy.io
curr_dir = os.getcwd()
# os.chdir('/home/thijs/repos/fishualizer/')
# from utilities import Zecording
# import pipeline_functions as fish_pf
os.chdir('/home/thijs/repos/dnp-code/PGM7/')
import rbm7
import utilities7 as utilities
import pickle
import rbm7_trackprogress
import rbm7_trackprogress_wb
os.chdir('/home/thijs/repos/dnp-code/PGM3_correct/source/')
import rbm
from fishualizer_utilities import Zecording
os.chdir(curr_dir)
import rbm_pipeline_functions as pf

/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "numba_utilities7.py", line 296:
@njit(signature0,parallel=True)
def erf_times_gauss_numba0(x):
^

  self.func_ir.loc))
/home/thijs/repos/dnp-code/PGM7/numba_utilities7.py:703: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 2d, A))
  mean_V = np.dot(weights,V)/sum_weights
/home/thijs/repos/dnp-code/PGM7/numba_utilities7.py:862: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, F), array(float32, 2d, A))
  dmean_v_dw = np.dot(s1.T,V)
/home/thijs/repos/dnp-code/PGM7/numba_utilities7.py:863: NumbaPerformanceWarning: np.d

In [2]:
## Plot settings

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Dejavu Sans'
plt.rcParams['font.style'] = 'normal'
plt.rcParams['lines.linewidth'] = 3
dr_colors = {'glm': '#008B8B', 'pca': '#808000', 'rbm': '#800080', 'fa': 'red', 'ica':'#157bf9'}
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color'] # default color cycle


dr_legend = {'pca': 'PCA', 'rbm': 'RBM', 'fa': 'FA', 'ica': 'ICA', 'glm': 'GLM'}

In [3]:
base_path = '/home/thijs/Desktop/'
data_sets = {#'20180912-Run01': '20180912_Run01Tset=.h5'}
            '20180912-Run01': '20180912_Run01_spontaneous_rbm2.h5'}#,

## Load data

time_cutoff_end = 4000 # hard coded for now
# time_cutoffs = {''}
recordings = {}
for data_set, data_path in data_sets.items():
    recordings[data_set] = Zecording(path=base_path + data_path, kwargs={'ignorelags': True,
                                              'forceinterpolation': False,
                                              'ignoreunknowndata': True,# 'parent': self,
                                              'loadram': True})  # load data
rec = recordings['20180912-Run01']
# rec = recordings['2019-03-26(Run09)']
print(rec)
regions = {#'rh1': np.array([218]), 'rhall': np.array([113]),
          'wb': np.arange(294)}
selected_neurons = {}
n_sel_cells = {}
train_data = {}
test_data = {}
full_data = {}
train_times = rec.times[:time_cutoff_end]
test_times = rec.times[time_cutoff_end:]

for ir in list(regions.keys()):
    selected_neurons[ir] = np.unique(scipy.sparse.find(rec.labels[:, regions[ir]])[0])
    n_sel_cells[ir] = len(selected_neurons[ir])
    train_data[ir] = rec.spikes[selected_neurons[ir], :time_cutoff_end]
    test_data[ir] = rec.spikes[selected_neurons[ir], time_cutoff_end:]
    full_data[ir] = rec.spikes[selected_neurons[ir], :]

/home/thijs/.conda/envs/py37/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
baseline with shape (5553, 54334) is not recognized, so it cannot be loaded.
drifts with shape (5553, 2) is not recognized, so it cannot be loaded.
inferredspikes with shape (5553, 54334) is not recognized, so it cannot be loaded.
ljpcoordinates with shape (3, 54334) is not recognized, so it cannot be loaded.
segmentation with shape (30, 598, 1280) is not recognized, so it cannot be loaded.
temporalmean with shape (30, 598, 1280) is not recognized, so it cannot be loaded.
rawsignal with shape (5553, 54334) is not recognized, so it cannot be loaded.
trace with shape (5553, 1) is not recognized, so it cannot be loaded.
metadata with shape (1, 1) is not recognized, so it cannot be loaded.


Recording from /home/thijs/Desktop/20180912_Run01_spontaneous_rbm2.h5


In [4]:
inds_wb = np.unique(scipy.sparse.find(rec.labels[:, np.arange(294)])[0])
# inds_rhall = np.unique(scipy.sparse.find(rec.labels[:, 113])[0])
# inds_rh1_of_all = scipy.sparse.find(rec.labels[:, 218])[0]
# inds_rh1 = np.arange(len(np.unique(scipy.sparse.find(rec.labels[:, 218])[0])))
# inds_rh1_of_wb = scipy.sparse.find(rec.labels[inds_wb, 218])[0]
# inds_rh1_of_rhall = scipy.sparse.find(rec.labels[inds_rhall, 218])[0]
# inds_rhall_of_wb = scipy.sparse.find(rec.labels[inds_wb, 113])[0]
# np.median(llh_array['2019-05-10-2036'][inds_rh1_of_rhall])

# inds_rh1_all = {'rh1': inds_rh1, 'rhall': inds_rh1_of_rhall, 'wb': inds_rh1_of_wb}
# inds_all_dict = {'rh1': inds_rh1, 'rhall': inds_rhall, 'wb': inds_wb}

In [5]:
# selected_neurons = scipy.sparse.find(rec.labels[:, 218])[0]
tmp_reg = 'wb'

population_rate = np.sum(rec.spikes[selected_neurons[tmp_reg], :], axis=0)
sorted_pr = np.sort(population_rate)
pr_coords = np.vstack((range(len(population_rate)), sorted_pr)).T
first_point = pr_coords[0, :]
line_vector = pr_coords[-1, :] - first_point
line_vector_norm = line_vector / np.sqrt(np.sum(line_vector ** 2))
vector_fromfirst = pr_coords - first_point
# take vector product of vector_fromfirst * line_vector_norm to get parallel vector:
scalar_product = np.sum(vector_fromfirst * 
                        np.matlib.repmat(line_vector_norm, len(population_rate), 1), axis=1)
# now find perpendicular vector:
vector_fromfirst_par = np.outer(scalar_product, line_vector_norm)
vector_to_line = vector_fromfirst - vector_fromfirst_par
distance_to_line = np.sqrt(np.sum(vector_to_line ** 2, axis=1))

burst_cutoff = sorted_pr[np.argmax(distance_to_line)]
pop_rate_train = np.sum(rec.spikes[selected_neurons[tmp_reg], :time_cutoff_end], axis=0)
ind_train_wo_bursts = pop_rate_train <= burst_cutoff
pop_rate_test = np.sum(rec.spikes[selected_neurons[tmp_reg], time_cutoff_end:], axis=0)
ind_test_wo_bursts = pop_rate_test <= burst_cutoff
pop_rate_full = np.sum(rec.spikes[selected_neurons[tmp_reg], :], axis=0)
ind_full_wo_bursts = pop_rate_full <= burst_cutoff
# print(burst_cutoff)
print(f'Fraction not burst: {np.sum(population_rate <= burst_cutoff) / len(population_rate)}')

Fraction not burst: 0.892670628489105


In [11]:
burst_cutoff / len(selected_neurons['wb'])


0.08770326364294147